In [1]:
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
import numpy as np

import sys
sys.path.insert(0,'..')

from rcalgo.tf.utils.text_utils import *
import pickle as pkl
from rcalgo.tf.tfmodels.texar_models.tx_transformer_seq2seq import *
#import texar as tx
from scipy.stats import rv_discrete
import texar.tf as tx
#from texar_models.tx_rnnlm import *


In [2]:
with open('tmp_photo_comments_sentences.pkl','rb') as f:
    sentences = pkl.load(f)
word_freq = word_count(sentences, word_seg=False)
term_id_map = build_word2idx(word_freq, freq_threshold=5)
id_term_map = dict([(v,k) for k, v in term_id_map.items()]) 


In [3]:
#term_id_map = pkl.load(open('word_dict.pkl', 'rb'))
#id_term_map = dict([(v,k) for k, v in term_id_map.items()]) 


In [3]:
from rcalgo.tf.utils.train_config import *

args = parse_initial_training_args(
        [
            '--learning-rate', '0.0001',
            '--n-epochs', '5',
            '--opt', 'adam',
            '--batch-size', '512',
            '--p-coef', '0',
            '--keep-prob','1.0',
            '--embedding-size', '512',
            '--model-dir', 'model/',
            '--model-name', 'masked_seq2seq',
            '--num-layers', '2',
            '--gpu-id', '0',
            '--gpu-num', '4',
            '--hidden-size','512',
            '--maxlen', '50',
            '--epoch-save',
            '--beam-width', '200',
            '--expand-input',
            '--n-head', '8',
            '--position-type','2,3,4,5',
            '--transformer-func','relu',
            '--conv-num-filters', '512',
            '--low-sent-scale-length', '4',
            '--high-sent-scale-length', '8',
            '--position-info'
        ]
    )
args.add_conv_flag = False
args.output_maxlen = args.maxlen

In [ ]:
res = {}
graph_to_use = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
#rnnlm_model = TxRNNLMModel(args, graph_to_use, word_dict=term_id_map)
rnnlm_model = TxMaskedTransformerSeq2SeqModel(args, graph_to_use, word_dict=term_id_map)
rnnlm_model.build_model()
rnnlm_model.build_model_summary()
#rnnlm_model.job.dump_checkpoint()
display(rnnlm_model.model_summary())
rnnlm_model.run([sentences], test_size=0.2)

total parameter number: 24755200


variable_name  variable_shape  \
0     gpu/embedding/embedding_layer/embedding_table:0     [6178, 512]   
1            gpu/mw_conv_block/conv_1d/he_uniform/W:0   [2, 512, 512]   
2                       gpu/mw_conv_block/conv_1d/B:0           [512]   
3          gpu/mw_conv_block/conv_1d_1/he_uniform/W:0   [3, 512, 512]   
4                     gpu/mw_conv_block/conv_1d_1/B:0           [512]   
5          gpu/mw_conv_block/conv_1d_2/he_uniform/W:0   [4, 512, 512]   
6                     gpu/mw_conv_block/conv_1d_2/B:0           [512]   
7          gpu/mw_conv_block/conv_1d_3/he_uniform/W:0   [5, 512, 512]   
8                     gpu/mw_conv_block/conv_1d_3/B:0           [512]   
9                 gpu/pos_embedding/embedding_table:0       [50, 512]   
10  gpu/transform2_block/multi_head_attention/conv...  [1, 512, 1536]   
11  gpu/transform2_block/multi_head_attention/conv...          [1536]   
12  gpu/transform2_block/multi_head_attention/conv...   [1, 512, 512]   
13  gpu/transform2_block/multi_head_attention/conv...           [512]   
14               gpu/transform2_block/ln_layer/beta:0           [512]   
15              gpu/transform2_block/ln_layer/gamma:0           [512]   
16  gpu/transform2_block/ffn_mlp/conv_1d/he_unifor...  [1, 512, 2048]   
17           gpu/transform2_block/ffn_mlp/conv_1d/B:0          [2048]   
18  gpu/transform2_block/ffn_mlp/conv_1d_1/he_unif...  [1, 2048, 512]   
19         gpu/transform2_block/ffn_mlp/conv_1d_1/B:0           [512]   
20             gpu/transform2_block/ln_layer_1/beta:0           [512]   
21            gpu/transform2_block/ln_layer_1/gamma:0           [512]   
22  gpu/transform2_block_1/multi_head_attention/co...  [1, 512, 1536]   
23  gpu/transform2_block_1/multi_head_attention/co...          [1536]   
24  gpu/transform2_block_1/multi_head_attention/co...   [1, 512, 512]   
25  gpu/transform2_block_1/multi_head_attention/co...           [512]   
26             gpu/transform2_block_1/ln_layer/beta:0           [512]   
27            gpu/transform2_block_1/ln_layer/gamma:0           [512]   
28  gpu/transform2_block_1/ffn_mlp/conv_1d/he_unif...  [1, 512, 2048]   
29         gpu/transform2_block_1/ffn_mlp/conv_1d/B:0          [2048]   
..                                                ...             ...   
44  gpu/transformer_decoder/layer_0/encdec_attenti...      [512, 512]   
45  gpu/transformer_decoder/layer_0/encdec_attenti...      [512, 512]   
46  gpu/transformer_decoder/layer_0/encdec_attenti...      [512, 512]   
47  gpu/transformer_decoder/layer_0/past_poswise_l...           [512]   
48  gpu/transformer_decoder/layer_0/past_poswise_l...           [512]   
49  gpu/transformer_decoder/layer_0/ffn/conv1/kern...     [512, 2048]   
50   gpu/transformer_decoder/layer_0/ffn/conv1/bias:0          [2048]   
51  gpu/transformer_decoder/layer_0/ffn/conv2/kern...     [2048, 512]   
52   gpu/transformer_decoder/layer_0/ffn/conv2/bias:0           [512]   
53             gpu/transformer_decoder/layer_1/beta:0           [512]   
54            gpu/transformer_decoder/layer_1/gamma:0           [512]   
55  gpu/transformer_decoder/layer_1/self_attention...      [512, 512]   
56  gpu/transformer_decoder/layer_1/self_attention...      [512, 512]   
57  gpu/transformer_decoder/layer_1/self_attention...      [512, 512]   
58  gpu/transformer_decoder/layer_1/self_attention...      [512, 512]   
59  gpu/transformer_decoder/layer_1/encdec_attenti...           [512]   
60  gpu/transformer_decoder/layer_1/encdec_attenti...           [512]   
61  gpu/transformer_decoder/layer_1/encdec_attenti...      [512, 512]   
62  gpu/transformer_decoder/layer_1/encdec_attenti...      [512, 512]   
63  gpu/transformer_decoder/layer_1/encdec_attenti...      [512, 512]   
64  gpu/transformer_decoder/layer_1/encdec_attenti...      [512, 512]   
65  gpu/transformer_decoder/layer_1/past_poswise_l...           [512]   
66  gpu/transformer_decoder/layer_1/past_poswise_l...           [512]   
67  gpu/transformer_decod

total number of epochs: 5
tasks: [<tftraining.incremental_training.Task object at 0x7fe4b9e7ee80>]
lr values: [1e-04]


In [5]:
with rnnlm_model.job.session.graph.as_default():
    #print(tf.get_collection('input_dict'))
    input_var, masked_flag, ori_bow, indices, masked_bow, input_context = tf.get_collection('input_dict')
    go_value = tf.get_collection('go_value')[0]
    outputs = tf.get_collection('output_dict')[0:5]
    greedy_logits, greedy_sample_id, sample_logits, sample_sample_id, beam_search_results = outputs
    top_k = tf.get_collection('top_k_sample')[0]
    softmax_temperature = tf.get_collection('softmax_temperature')[0]


In [6]:
def out_print(num_arr, id_term_map, eos=rnnlm_model.nb_words - 1):
    #print(num_arr)
    text = ''
    for i in num_arr:
        if i >= eos-100:
            break
        else:
            text += id_term_map[i]
    return text

In [17]:
sent = np.random.choice(sentences, 1)
idxs,samples,beam_samples = rnnlm_model.job.session.run([indices, sample_sample_id, beam_search_results], 
                                                         feed_dict={input_var:sent})
replace_str = ''.join([sent[0][i-1] for i in idxs[:,1] if i < len(sent[0])])


print(sent[0])
print()
print(replace_str)
print()
print(sent[0].replace(replace_str, out_print(samples[0][1:], id_term_map)))
print()

厉害到人都打不死😂

人都

厉害到你都打不死😂



In [18]:
samples

array([[6176,   22,  149, 6178, 6178, 6178, 6178, 6178, 6178, 6178, 6178,
        6178, 6178, 6178, 6178]], dtype=int32)

In [ ]:
#sent = np.random.choice(sentences, 1)
sent = ['今天天气不错']
bow = rnnlm_model.job.session.run(ori_bow, feed_dict={input_var:sent})

In [ ]:
bow[0,5:7]=31255

In [ ]:
s1, s2 = rnnlm_model.job.session.run([sample_sample_id], feed_dict={ori_bow:bow, masked_flag:False})


In [ ]:
s1, s2 = rnnlm_model.job.session.run([sample_sample_id,beam_search_results], feed_dict={ori_bow:bow, masked_flag:False})


In [ ]:
print(out_print(s1[0], id_term_map))
results = set([out_print(s2[0][:,i], id_term_map) for i in range(200)])
for i in results:
    print(i)

In [ ]:
samples = rnnlm_model.job.session.run(sample_sample_id, feed_dict={input_var:  
                                                                   top_k:10,
                                                                   masked_flag:False })
print(sent)
print(out_print(samples[0], id_term_map))


In [ ]:
samples = rnnlm_model.job.session.run(beam_search_results, feed_dict={ori_bow:bow, 
                                                                      top_k:10,
                                                                      masked_flag:False })
print(sent)
print(out_print(samples[0], id_term_map))


In [ ]:
bow[0][5:7] = 8174

In [ ]:
samples = rnnlm_model.job.session.run(beam_search_results, feed_dict={ori_bow: bow, 
                                                                      top_k:10,
                                                                      masked_flag:False })
print(sent)
for i in range(20):
    print(out_print(samples[0][:,i], id_term_map))

In [ ]:
sent = np.random.choice(sentences, 1)
#sent = ['天气还可以']
samples = rnnlm_model.job.session.run(beam_search_results, feed_dict={input_var:sent, 
                                                                      top_k:10,
                                                                      masked_flag:True })
print(sent)
#for i in range(200):
results = set([out_print(samples[0][:,i], id_term_map) for i in range(200)])
for i in results:
    print(i)
